In [1]:
!

arrays.npy	extract_dbn.py	      modified_dbn.py  svm_test.py
belief.py	extract_knowledge.py  modified_rbm.py  test_datasets.py
datalab		inference.py	      __pycache__      utils.py
dbn_extract.py	insert_knowledge.py   rbm.py	       yale_dataset
dbn.py		MNIST_data	      rule.py	       yale_dataset.zip


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [0]:
uploaded = drive.CreateFile({'parents':[{u'id': "1HnaZE4RY-8iJkPjwfXwd_kTBjewKvDZs"}]})
uploaded.SetContentFile('arrays.npy')
uploaded.Upload()

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
import extract_knowledge

from tensorflow.examples.tutorials.mnist import input_data
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from rbm import RBM
from utils import tile_raster_images
from inference import quantitativeInference

In [0]:
def test_mnist(with_rules=False, one_hot=False, hidden_units=500, n_test=1):
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=one_hot)
    trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images,\
        mnist.test.labels

    outputs = []
    how_many = 10000
    num_classes = 10
    accs = []
    wr_str = str(with_rules)
    hu_str = str(hidden_units)
    filename = "mnist_" + wr_str + "_" + hu_str + ".npy"

    for t in range(n_test):
        print("\n--------- TEST {} WITH{} RULES---------\n".format(t, "" if with_rules==True else "OUT"))
        rbm = RBM(trX.shape[1], hidden_units)
        rbm.train(trX, debug=False)
        print("!!! RBM trained !!!")

        if with_rules:
            rules_rbm = extract_knowledge.rbm_extract(rbm.w)
            print("Calculating beliefs...")
            new_trX = [quantitativeInference([rules_rbm], x) for x in trX]
            new_teX = [quantitativeInference([rules_rbm], x) for x in teX]
        else:
            input_ = tf.placeholder("float", [None, rbm._input_size])
            w_ = tf.placeholder("float", [rbm._input_size, rbm._output_size])
            hb_ = tf.placeholder("float", [rbm._output_size])
            with tf.Session() as sess:
                new_trX = sess.run(rbm.foward_pass(input_, w_, hb_), feed_dict={
                    input_: trX, w_: rbm.w, hb_: rbm.hb})
                new_teX = sess.run(rbm.foward_pass(input_, w_, hb_), feed_dict={
                    input_: teX, w_: rbm.w, hb_: rbm.hb})

        svm_inf = SVC()
        print("Fitting...")
        svm_inf.fit(new_trX, trY)
        print("Inferring...")
        pred = svm_inf.predict(new_teX)
        acc = (np.sum(pred == teY) / teY.shape[0]) * 100
        print("Accuracy: {}".format(acc))
        accs.append(acc)

        # Saving accuracy values
        accs_np = np.asarray(accs)
        np.save(filename, accs_np)
        
        # Saving on google drive
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default() 
        drive = GoogleDrive(gauth)
        
        uploaded = drive.CreateFile({'parents':[{u'id': "1HnaZE4RY-8iJkPjwfXwd_kTBjewKvDZs"}]})
        uploaded.SetContentFile(filename)
        uploaded.Upload()

In [0]:
def test_yale(with_rules=False, hidden_units=500, n_test=1):
    dataset = []
    labels = []
    accs = []
    wr_str = str(with_rules)
    hu_str = str(hidden_units)
    filename = "yale_" + wr_str + "_" + hu_str + ".npy"

    # Preparing dataset
    for i in range(1, 16):
        filelist = glob.glob('./yale_dataset/subject'+str(i).zfill(2)+"*")
        for filename in filelist:
            img = Image.open(filename).convert('L')
            img = img.resize((132, 132))
            img = np.array(img, dtype=np.float32)
            img = np.reshape(img, [img.shape[0]*img.shape[1]])
            img = img / 255.
            dataset.append(img)
            labels.append(float(i))

    dataset = np.asarray(dataset)
    labels = np.asarray(labels)

    for t in range(n_test):
        X_train, X_test, y_train, y_test = train_test_split(
            dataset, labels, test_size=31, random_state=42, stratify=labels)

        print("\n--------- TEST {} WITH{} RULES---------\n".format(t, "" if with_rules==True else "OUT"))
        rbm = RBM(X_train.shape[1], with_rules)
        rbm.train(dataset, debug=True, epochs=500, batchsize=11, learning_rate=0.1)
        print("!!! RBM trained !!!")

        if with_rules:
            rules_rbm = extract_knowledge.rbm_extract(rbm.w)
            print("Calculating beliefs...")
            new_trX = [quantitativeInference([rules_rbm], x) for x in X_train]
            new_teX = [quantitativeInference([rules_rbm], x) for x in X_test]
        else:
            input_ = tf.placeholder("float", [None, rbm._input_size])
            w_ = tf.placeholder("float", [rbm._input_size, rbm._output_size])
            hb_ = tf.placeholder("float", [rbm._output_size])
            with tf.Session() as sess:
                new_trX = sess.run(rbm.foward_pass(input_, w_, hb_), feed_dict={
                                    input_: X_train, w_: rbm.w, hb_: rbm.hb} )
                new_teX = sess.run(rbm.foward_pass(input_, w_, hb_), feed_dict={
                                    input_: X_test, w_: rbm.w, hb_: rbm.hb} )

        svm_inf = SVC()
        print("Fitting...")
        svm_inf.fit(new_trX, y_train)
        print("Inferring...")
        pred = svm_inf.predict(new_teX)
        acc = (np.sum(pred == y_test) / y_test.shape[0]) * 100
        print("Accuracy: {}".format(acc))
        accs.append(acc)

        # Saving accuracy values
        accs_np = np.asarray(accs)
        np.save(filename, accs_np)
        
        # Saving on google drive
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default() 
        drive = GoogleDrive(gauth)
        
        uploaded = drive.CreateFile({'parents':[{u'id': "1HnaZE4RY-8iJkPjwfXwd_kTBjewKvDZs"}]})
        uploaded.SetContentFile(filename)
        uploaded.Upload()

In [0]:
# Testing mnist
test_mnist(with_rules=False, hidden_units=500, n_test=10)
test_mnist(with_rules=False, hidden_units=1000, n_test=10)
test_mnist(with_rules=True, hidden_units=500, n_test=10)
test_mnist(with_rules=True, hidden_units=1000, n_test=10)

# # Testing yale
test_yale(with_rules=False, hidden_units=500, n_test=10)
test_yale(with_rules=False, hidden_units=1000, n_test=10)
test_yale(with_rules=True, hidden_units=500, n_test=10)
test_yale(with_rules=True, hidden_units=1000, n_test=10)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

--------- TEST 0 WITHOUT RULES---------

